In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import numpy

In [4]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
post_list = []

In [6]:
with open("pinterestData.txt","r") as file:
    dataList = file.readlines()
dataList = [item.strip() for item in dataList]
post_list.extend(dataList)

FileNotFoundError: [Errno 2] No such file or directory: 'pinterestData.txt'

In [ ]:
with open("facebookPost.txt","r", encoding="utf-8") as file:
    dataList = file.readlines()
dataList = [item.strip() for item in dataList]
post_list.extend(dataList)

In [7]:
with open("redditPost.txt","r", encoding="utf-8") as file:
    dataList = file.readlines()
dataList = [item.strip() for item in dataList]
post_list.extend(dataList)

In [8]:
with open("keyword.txt","r") as file:
    hashtag_to_search = file.read().strip()

In [9]:
list_keyword = []
def extract_topics(posts, num_topics = 5):
    vectorizer = TfidfVectorizer(max_df=0.85, max_features=1000, stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(posts)
    
    nmf = NMF(n_components=num_topics, random_state=42)
    nmf.fit(tfidf_matrix)
    
    feature_names = vectorizer.get_feature_names_out()
    for i,topic in enumerate(nmf.components_):
        top_words_idx = topic.argsort()[:-10-1:-1]
        top_words = [feature_names[idx] for idx in top_words_idx]
        list_keyword.append(" ".join(top_words[0:2]))
extract_topics(post_list)
list_keyword

['christmas boys',
 'merry christmas',
 'aww christmasclub',
 'pics happyhungryhijabi',
 'practicalday9048 christmaslights']

In [10]:
def get_bert_embeddings(posts):
    model_name = 'bert-base-uncased'
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)

    if isinstance(posts, str):
        posts = [posts]
    
    encoded_posts = tokenizer(posts, return_tensors='pt', padding = True, truncation=True)

    with torch.no_grad():
        outputs = model(**encoded_posts)
        embeddings = outputs['last_hidden_state'][:,0, :].numpy()
    return embeddings

def compute_cosine_similarity(post_embeddings, topic_embedding):
    post_embeddings = post_embeddings.reshape(post_embeddings.shape[0],-1)
    topic_embedding = topic_embedding.reshape(1, -1)
    similarities = cosine_similarity(post_embeddings, topic_embedding)
    return similarities.flatten()

def find_relevance_to_topic(posts, topic, threshold = 0.5):
    post_embeddings = get_bert_embeddings(posts)
    topic_embedding = get_bert_embeddings(topic)
    similarities = compute_cosine_similarity(post_embeddings,topic_embedding)
    relevant_posts = [post for post, sim in zip(posts, similarities) if sim > threshold]
    return relevant_posts

relevant_posts = find_relevance_to_topic(post_list,hashtag_to_search)
relevant_posts

['Christmas on more people : Christmas tree. My christmas :',
 'Christmas boys, Christmas…',
 'Oh, Christmas Cat oh Christmas cat…',
 'Grandpa won Christmas!',
 'Christmas lights and christmas market, first ever street photography session.',
 '“Christmas with DOOM” a virtual christmas card',
 'Christmas',
 'Celebrate Christmas With Creative Christmas Trees!!🎄🎄❤️💚',
 'Christmas',
 'Christmas wishlist :',
 '"No, he\'s the Christmas Rapist! He rapes on Christmas! Is nothing sacred?!"',
 'Merry Christmas!!!⛄🎅🦌',
 "It's a Christmas miracle",
 'Magical Christmas Fireplace • 10-Hour Christmas Ambience',
 'Spending Christmas away from family this year…..merry Christmas to me!',
 'Christmas humour',
 'New Christmas Tree + Christmas decorations + Chinese NY decorations Leaked by Clash Kurdish',
 'Christmas Blessings',
 'Christmas 2023',
 'Christmas time',
 'Christmas Gifting',
 'Christmas came early',
 'r/christmas',
 'r/ChristmasDecorating',
 'r/ChristmasLights',
 'r/pics',
 'r/aww',
 'u/Christ

In [ ]:
if(len(relevant_posts) > 10):
    relevant_posts = relevant_posts[:10]
if(len(list_keyword) > 5) : 
    list_keyword = list_keyword[:5]

In [ ]:
with open("post.txt","w", encoding="utf-8") as file:
    for post in relevant_posts:
        file.write(str(post) + "\n")

In [ ]:
with open("postKeyword.txt","w", encoding="utf-8") as file:
    for post in list_keyword:
        file.write(str(post) + "\n")